<a href="https://colab.research.google.com/github/sohail-datascientist/sohail-datascientist/blob/main/VGG16_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Essential Librarries Added for the functionalities.

In [1]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision as tv
import torch
import cv2 as cv
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn

**Transformed Data to tensor and normalized it with different params**

In [3]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((1.5, 2.5, 0.5), (1, 0.25, 0.15))
    ]
)

**CIFAR10 Dataset trainig loader and test along with train set and test set chosen from cifar10 dataset**

In [4]:
train_set = ds.CIFAR10(root='../input/', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0)
testset = tv.datasets.CIFAR10(root='../input/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset,  batch_size=4, shuffle=True, num_workers=0)


Extracting ../input/cifar-10-python.tar.gz to ../input/
Files already downloaded and verified


Classes of the dataset

In [5]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

ModelNamed My_VGG_16

Sequential 


In [6]:
class My_VGG_16(nn.Module):
    def __init__(self, num_classes):
        super(My_VGG_16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), 
            nn.ReLU(True),
            nn.Conv2d(32, 32, 3, padding=1), 
            nn.ReLU(True),
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(32, 64, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  
            nn.Conv2d(64, 128, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(128, 128, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(128, 128, 3, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(128, 256, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),  
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1), 
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.ReLU(True), 
        )
        self.classifier = nn.Sequential(
            nn.Linear(2 * 2 * 256, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [7]:
net = My_VGG_16(100)
net.cuda()
lr = 1e-3
momentum = 0.08
num_epoch = 10

critierion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
print('Training with learning rate = %f, momentum = %f ' % (lr, momentum))

Training with learning rate = 0.001000, momentum = 0.080000 


In [8]:
loss_p = np.array([])
for t in range(num_epoch):
    running_loss = 0
    running_loss_sum_per_epoch = 0
    total_images = 0
    correct_images = 0
    if t == 25:
        optimizer = optim.SGD(net.parameters(), lr=lr/10, momentum=momentum)
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = net(images)
        _, predicts = torch.max(outputs.data, 1)
        loss = critierion(outputs, labels)

        loss.backward()

        optimizer.step()

        total_images += labels.size(0)
        correct_images += (predicts == labels).sum().item()
        loss_data = loss.data.item()
        running_loss += loss_data
        
        running_loss_sum_per_epoch = running_loss + running_loss_sum_per_epoch
        if i % 2000 == 1999:
            print('Epoch, batch [%d, %5d] loss: %.6f, Training accuracy: %.5f' %
                  (t + 1, i + 1, running_loss / 2000, 100 * correct_images / total_images))
            running_loss = 0
            total_images = 0
            correct_images = 0

    loss_p = np.append(loss_p, running_loss_sum_per_epoch)

print('Finished training.')


Epoch, batch [1,  2000] loss: 3.529600, Training accuracy: 10.06250
Epoch, batch [1,  4000] loss: 2.384281, Training accuracy: 10.11250
Epoch, batch [1,  6000] loss: 2.360781, Training accuracy: 10.22500
Epoch, batch [1,  8000] loss: 2.354773, Training accuracy: 9.55000
Epoch, batch [1, 10000] loss: 2.344983, Training accuracy: 10.08750
Epoch, batch [1, 12000] loss: 2.337720, Training accuracy: 10.58750
Epoch, batch [2,  2000] loss: 2.336126, Training accuracy: 10.02500
Epoch, batch [2,  4000] loss: 2.333348, Training accuracy: 10.10000
Epoch, batch [2,  6000] loss: 2.330898, Training accuracy: 10.61250
Epoch, batch [2,  8000] loss: 2.331087, Training accuracy: 10.47500
Epoch, batch [2, 10000] loss: 2.326917, Training accuracy: 10.40000
Epoch, batch [2, 12000] loss: 2.291014, Training accuracy: 11.92500
Epoch, batch [3,  2000] loss: 2.181705, Training accuracy: 16.26250
Epoch, batch [3,  4000] loss: 2.150098, Training accuracy: 16.66250
Epoch, batch [3,  6000] loss: 2.131373, Training 

In [9]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(testloader):
        test_images_set = test_images_set.cuda()
        test_labels_set = test_labels_set.cuda()
    
        y_predicted = net(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 61.150000000000006%
